In [1]:
import sys
path = ".."
if path not in sys.path:
    sys.path.insert(0, path)

In [ ]:
from data_retrieval import lipade_groundtruth
from absolute_path import absolutePath
from lavis.models import load_model_and_preprocess
from PIL import Image
from tqdm import tqdm
import torch
import csv
import os

corpus = "lipade_groundtruth"
device = "cpu" # "cuda" if torch.cuda.is_available() else "cpu"

model, vis_processors, _ = load_model_and_preprocess(
    name="blip_caption", model_type="large_coco", is_eval=True, device=device
)

model.to(device).half()

/home/samuel-g/.pyenv/versions/3.11.11/lib/python3.11/site-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
100%|██████████| 1.66G/1.66G [02:13<00:00, 13.4MB/s]
/home/samuel-g/.pyenv/versions/3.11.11/lib/python3.11/site-packages/lavis/models/base_model.py:40: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be al

BlipCaption(
  (visual_encoder): VisionTransformerEncoder(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 1024, kernel_size=(16, 16), stride=(16, 16))
      (norm): Identity()
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (blocks): ModuleList(
      (0): Block(
        (norm1): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=1024, out_features=3072, bias=True)
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=1024, out_features=1024, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (drop_path): Identity()
        (norm2): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (act): GELU(approximate='none')
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (drop): Dropout(p=0.0, inplace=False)
  

In [4]:
x,_,y = lipade_groundtruth.getDataset(mode="similar")
images = []
for i in range(len(x)):
    images.append(Image.open(x[i]).convert('RGB'))

In [8]:
image = images[0]
image = vis_processors["eval"](image).unsqueeze(0).to(device, torch.float16)
model.generate({"image": image})

['a drawing of a group of men standing in front of a train']

In [9]:
prompts = [
    "",
    "Question: What is represented in this photograph? Answer: ",
    "A photograph representing ",
]

In [ ]:
path = absolutePath + 'data_generation/generated/' + corpus + "/"

for prompt in prompts:
    for i in tqdm(range(len(x)), desc=prompt):
        image = images[i]
        filename = x[i].split('/')[-1].split('.')[0] + '.csv'

        with open(path + filename, mode='r') as infile:
            reader = csv.reader(infile)
            dict = {rows[0]: rows[1] for rows in reader}
        if prompt in dict:
            continue

        image = vis_processors["eval"](image).unsqueeze(0).to(device, torch.float16)
        if prompt == "":
            generated_text = model.generate({"image": image})
        else:
            generated_text = model.generate({"image": image, "prompt": prompt})
        dict[prompt] = generated_text[0].strip()

        with open(path + filename, 'w') as outfile:
            res_dict = csv.writer(outfile)
            kv = list(dict.items())
            for key, value in kv:
                res_dict.writerow([key, value])

100%|██████████| 262/262 [49:16:17<00:00, 677.01s/it]    
